In [1]:
import pandas as pd
import pyspark.pandas as ps
from pyspark.sql import SparkSession
import io
from config import *
from unidecode import unidecode
import re

c:\repositorio\desafionst\desafionst\lib\site-packages\pyspark\pandas\__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [2]:
# Crie uma SparkSession
spark = SparkSession.builder.appName("ETL").getOrCreate()

<p>CAMADA BRONZE</p>
Recebimento dos arquivos e transformação para parquet

Leitura do primeiro Arquivo

In [3]:
try:
    cargos = pd.read_csv('C:/repositorio/desafionst/bronze/BaseCargos.csv', delimiter=';', low_memory=False)
except pd.errors.ParserError as e:
    print(f"Erro na leitura do arquivo: {e}")

Erro na leitura do arquivo: Error tokenizing data. C error: Expected 8 fields in line 10, saw 9



Houve um erro de leitura do arquivo então, verifiquei o erro e foi ajustado.

In [4]:
#Como tentei abrir com o Pandas e deu erro, aqui eu utilizei a biblioteca io para ler o conteúdo e substituir o erro que era uma linha contendo ";" duas vezes e substitui para uma só
with open('C:/repositorio/desafionst/bronze/BaseCargos.csv', 'r', encoding='utf-8') as file:
    content = file.read()

content = content.replace(';;', ';')

In [5]:
#Aqui já acessando o arquivo ajustado(Utilizei a io.StringIO pois o conteúdo com o replace estava na memória por causa da substituição) 
try:
    df_cargos = pd.read_csv(io.StringIO(content), sep=';')
except pd.errors.ParserError as e:
    print(f"Erro na leitura do arquivo: {e}")

In [6]:
df_cargos

,Cargo,Nível,Área,COD Área,COD Nível,Quadro,Bonus,Contratacao
0,OPV,Diretor,Operações,JAJ,JE,Efetivo,S,Diretoria
1,LOI,Estagiário,Logísitca,EDE,JA,Efetivo,N,Gerente
2,ADI,Estagiário,Administrativo,BAC,JA,Efetivo,N,Gerente
3,ADII,Analista,Administrativo,BAC,DB,Terceiro,N,RH
4,OPII,Analista,Operações,JAJ,DB,Terceiro,N,RH
5,FIV,Diretor,Financeiro,CBB,JE,Efetivo,S,Diretoria
6,FIII,Analista,Financeiro,CBB,DB,Terceiro,N,RH
7,ADIII,Coordenador,Administrativo,BAC,DB,Terceiro,N,RH
8,LOIV,Gerente,Logísitca,EDE,ID,Efetivo,S,Diretoria
9,FII,Estagiário,Financeiro,CBB,JA,Efetivo,N,Gerente


<p>Para os outros arquivos irá ser feito da mesma forma, verificando se irá ter erro na leitura e fazendo suas correções. Abaixo eu vou deixar uma leitura e tratamento de cada arquivo por célula para a visualização ficar melhor. </p>

OBS:
<p>Visualize que na linha "18", o registro da coluna "Área" não está de acordo. Todos os ajustes necessários serão feitos na camada Silver <p>

In [7]:
# Base CEP: Leitura e tratamento
try:
    # Tentar ler diretamente do arquivo
    cep = pd.read_csv('bronze\BaseCEP.csv', delimiter='|', low_memory=False)
except pd.errors.ParserError as e:
    print(f"Erro na leitura do arquivo: {e} Iniciando Tratamento de dados")

# Em caso de erro, tentar outra abordagem
with open('bronze\BaseCEP.csv', 'r', encoding='utf-8') as file:
    content = file.read()
# Realizar algum tratamento em 'content' se necessário    
content = content.replace('||', '|')

# Ler usando io.StringIO
try:
    df_cep = pd.read_csv(io.StringIO(content), sep='|')
    print (f'Leitura do arquivo Concluída')
except pd.errors.ParserError as e:
    print(f"Erro na leitura do arquivo: {e}")

Erro na leitura do arquivo: Error tokenizing data. C error: Expected 3 fields in line 10, saw 4
 Iniciando Tratamento de dados
Leitura do arquivo Concluída


In [8]:
# Base Clientes: Leitura e tratamento
try:
    clients = pd.read_csv('bronze\BaseClientes.csv', delimiter=';', low_memory=False)
except pd.errors.ParserError as e:
    print(f"Erro na leitura do arquivo: {e} Iniciando Tratamento de dados")

with open('bronze\BaseClientes.csv', 'r', encoding='utf-8') as file:
    content = file.read()
content = content.replace(';;', ';')

try:
    df_clients = pd.read_csv(io.StringIO(content), sep=';')
    print ('Leitura do arquivo Concluída')
except pd.errors.ParserError as e:
    print(f"Erro na leitura do arquivo: {e}")

Erro na leitura do arquivo: Error tokenizing data. C error: Expected 12 fields in line 5, saw 13
 Iniciando Tratamento de dados
Leitura do arquivo Concluída


In [9]:
# Base Funcionários: Leitura e tratamento
try:
    employers = pd.read_csv('bronze\BaseFuncionarios.csv', delimiter='|', low_memory=False)
except pd.errors.ParserError as e:
    print(f"Erro na leitura do arquivo: {e} Iniciando Tratamento de dados")

with open('bronze\BaseFuncionarios.csv', 'r', encoding='utf-8') as file:
    content = file.read()
content = content.replace('||', '|')

try:
    df_employers = pd.read_csv(io.StringIO(content), sep='|')
    print ('Leitura do arquivo Concluída')
except pd.errors.ParserError as e:
    print(f"Erro na leitura do arquivo: {e}")

Leitura do arquivo Concluída


In [10]:
# Base Nível: Leitura e tratamento
try:
    
    employers = pd.read_csv('bronze\BaseNível.csv', delimiter='%', low_memory=False)
except pd.errors.ParserError as e:
    print(f"Erro na leitura do arquivo: {e} Iniciando Tratamento de dados")

with open('bronze\BaseNível.csv', 'r', encoding='utf-8') as file:
    content = file.read()
content = content.replace('%%', '%')

try:
    df_baselevels = pd.read_csv(io.StringIO(content), sep='%')
    print ('Leitura do arquivo Concluída')
except pd.errors.ParserError as e:
    print(f"Erro na leitura do arquivo: {e}")

Leitura do arquivo Concluída


In [11]:
# Base PQ: Leitura e tratamento
try:
    employers = pd.read_csv('bronze\BasePQ.csv', delimiter=';', low_memory=False)
except pd.errors.ParserError as e:
    print(f"Erro na leitura do arquivo: {e} Iniciando Tratamento de dados")

with open('bronze\BasePQ.csv', 'r', encoding='utf-8') as file:
    content = file.read()
content = content.replace(';', ';')

try:
    df_basepq = pd.read_csv(io.StringIO(content), sep=';')
    print ('Leitura do arquivo Concluída')
except pd.errors.ParserError as e:
    print(f"Erro na leitura do arquivo: {e}")

Leitura do arquivo Concluída


<p>CAMADA SILVER</P>

Todos os arquivos lidos, agora irão ser colocados na próxima camada Silver e transferidos para Parquet. Fazendo isso a performance e o custo de armazenamento irão ser otimizados. 
Posteriormente foram feitos outros tipos de tratamentos como: Verificação de tipos de dados, null values, relacionamentos e outros os ajustes necessários para obtenção dos Insights.

In [12]:
# Salva os dados na camada Silver como Parquet
try:
    df_cargos.to_parquet(r'silver\cargos.parquet', index=False)
    print("Arquivo de Cargos transferidos para a camada silver com sucesso.")
except Exception as e:
    print(f"Erro ao salvar Arquivo de Cargos na camada silver: {e}")

try:
    df_cep.to_parquet(r'silver\cep.parquet', index=False)
    print("Arquivo de CEP transferidos para a camada silver com sucesso.")
except Exception as e:
    print(f"Erro ao salvar Arquivo de CEP na camada silver: {e}")

try:
    df_clients.to_parquet(r'silver\clients.parquet', index=False)
    print("Arquivo de Clientes transferidos para a camada silver com sucesso.")
except Exception as e:
    print(f"Erro ao salvar Arquivo de Clientes na camada silver: {e}")

try:
    df_employers.to_parquet(r'silver\employers.parquet', index=False)
    print("Arquivo de Funcionários transferidos para a camada silver com sucesso.")
except Exception as e:
    print(f"Erro ao salvar Arquivo de Funcionários na camada silver: {e}")

try:
    df_baselevels.to_parquet(r'silver\baselevels.parquet', index=False)
    print("Arquivo Base Nível transferidos para a camada silver com sucesso.")
except Exception as e:
    print(f"Erro ao salvar Arquivo Base Nível na camada silver: {e}")

try:
    df_basepq.to_parquet(r'silver\basepq.parquet', index=False)
    print("Arquivo Base PQ transferidos para a camada silver com sucesso.")
except Exception as e:
    print(f"Erro ao salvar Arquivo Base PQ na camada silver: {e}")

Arquivo de Cargos transferidos para a camada silver com sucesso.
Arquivo de CEP transferidos para a camada silver com sucesso.
Arquivo de Clientes transferidos para a camada silver com sucesso.
Arquivo de Funcionários transferidos para a camada silver com sucesso.
Arquivo Base Nível transferidos para a camada silver com sucesso.
Arquivo Base PQ transferidos para a camada silver com sucesso.


In [13]:
# Foi visualizado anteriormente que um Registo no arquivo de Cargos não estava certo, então vamos iniciar o tratamento com a função replace
df_cargos_parquet = pd.read_parquet('silver\cargos.parquet')
df_cargos_parquet.loc[df_cargos_parquet['Cargo'] == 'OPI', 'Área'] = df_cargos_parquet.loc[df_cargos_parquet['Cargo'] == 'OPI', 'Área'].str.replace('@@@Operações', 'Operações')
df_cargos_parquet

,Cargo,Nível,Área,COD Área,COD Nível,Quadro,Bonus,Contratacao
0,OPV,Diretor,Operações,JAJ,JE,Efetivo,S,Diretoria
1,LOI,Estagiário,Logísitca,EDE,JA,Efetivo,N,Gerente
2,ADI,Estagiário,Administrativo,BAC,JA,Efetivo,N,Gerente
3,ADII,Analista,Administrativo,BAC,DB,Terceiro,N,RH
4,OPII,Analista,Operações,JAJ,DB,Terceiro,N,RH
5,FIV,Diretor,Financeiro,CBB,JE,Efetivo,S,Diretoria
6,FIII,Analista,Financeiro,CBB,DB,Terceiro,N,RH
7,ADIII,Coordenador,Administrativo,BAC,DB,Terceiro,N,RH
8,LOIV,Gerente,Logísitca,EDE,ID,Efetivo,S,Diretoria
9,FII,Estagiário,Financeiro,CBB,JA,Efetivo,N,Gerente


Observe que os nomes das colunas estão em um formato não muito usual, com caracteres especiais, letras maiúsculas e minúsculas... Sem nenhum padrão.<br> 
Abaixo foi criado uma função para ajustar os nomes das colunas, remoção dos caracteres especiais e por fim a persistência no DataFrame para todos os arquivos.

In [14]:
# Lendo todos os arquivos para iniciar as tranformações necessárias e limpeza dos dados
df_cep_parquet = pd.read_parquet('silver\cep.parquet')
df_clients_parquet = pd.read_parquet('silver\clients.parquet')
df_employers_parquet = pd.read_parquet('silver\employers.parquet')
df_base_levels_parquet = pd.read_parquet(r'silver\baselevels.parquet')
df_base_pq_parquet = pd.read_parquet(r'silver\basepq.parquet')

In [15]:
def remover_caracteres_especiais(texto):
    texto_sem_acentos = unidecode(texto)
    return re.sub(r'[^a-zA-Z0-9\s]', '', texto_sem_acentos)

def formatar_nome_coluna(nome):
    nome_sem_especiais = remover_caracteres_especiais(nome)
    return nome_sem_especiais.lower().replace(' ', '_')

def formatacao_final(df):
    novos_nomes = [formatar_nome_coluna(col) for col in df.columns]
    df.columns = novos_nomes

formatacao_final(df_cargos_parquet)
formatacao_final(df_cep_parquet)
formatacao_final(df_clients_parquet)
formatacao_final(df_employers_parquet)
formatacao_final(df_base_levels_parquet)
formatacao_final(df_base_pq_parquet)


<p>CEP</p>

Para o CEP foram feitas validações de dados Nulos, e de integridade para a coluna 'cep'

In [16]:
df_cep_parquet

,cep,estado,regiao
0,20125535,Mato Grosso,Centro - Oeste
1,25995770,Rio Grande do Norte,Nordeste
2,37278465,Sergipe,Nordeste
3,49897703,Mato Grosso,Centro - Oeste
4,30149335,Mato Grosso,Centro - Oeste
...,...,...,...
855,801388803,Bahia,Nordeste
856,131504555,Acre,Norte
857,560973100,Roraima,Norte
858,233130568,Alagoas,Nordeste


In [17]:
# Verificação de registros nulos por Coluna
registros_nulos_total = df_cep_parquet.isnull().sum()
print(f"Total de registros:\n{registros_nulos_total}")

Total de registros:
cep       0
estado    0
regiao    0
dtype: int64


In [18]:
# Se tiver algum registro nulo na coluna CEP ele irá remover a linha
if df_cep_parquet['cep'].isnull().any():
    # Remover linhas com nulos na coluna 'cep'
    df_cep_parquet = df_cep_parquet.dropna(subset=['cep'])

df_cep_parquet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 860 entries, 0 to 859
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cep     860 non-null    int64 
 1   estado  860 non-null    object
 2   regiao  860 non-null    object
dtypes: int64(1), object(2)
memory usage: 20.3+ KB


In [19]:
# Verificação de Integridade
# Um CEP deve conter 8 dígitos, se for menor que isso ou maior podemos ter um problema, então um teste de integridade será feito para obter esse resultado

# Cria uma nova coluna "cep_valido" e verifica se os registros da coluna cep tem 8 digitos. Se tiver adiciona a True e se não, False.
df_cep_parquet['cep_valido'] = df_cep_parquet['cep'].astype(str).apply(lambda x: x.isdigit() and len(x) == 8)

# Contagem de valores True e False
contagem_validade_cep = df_cep_parquet['cep_valido'].value_counts()
contagem_validade_cep


cep_valido
True     559
False    301
Name: count, dtype: int64

In [20]:
df_cep_parquet

,cep,estado,regiao,cep_valido
0,20125535,Mato Grosso,Centro - Oeste,True
1,25995770,Rio Grande do Norte,Nordeste,True
2,37278465,Sergipe,Nordeste,True
3,49897703,Mato Grosso,Centro - Oeste,True
4,30149335,Mato Grosso,Centro - Oeste,True
...,...,...,...,...
855,801388803,Bahia,Nordeste,False
856,131504555,Acre,Norte,False
857,560973100,Roraima,Norte,False
858,233130568,Alagoas,Nordeste,False


E por fim, vamos transformar o dataframe em parquet e carregá-lo na camada GOLD

In [21]:
df_cep_parquet.to_parquet('gold/cep.parquet')

<p>Clientes<p>
Para os clientes vamos oberservar os mesmo aspectos: Integridade, Nulos e Limpeza


In [22]:
df_clients_parquet


,cliente,valor_contrato_anual,quantidade_de_servicos,cargo_responsavel,cep,data_inicio_contrato,nivel_de_importancia,unnamed_7,unnamed_8,unnamed_9
0,Teixeira Gonçalves,54000,37,ADIII,743419711.0,15/03/2019,3.0,NaN,NaN,NaN
1,Souza Santos,126000,12,FIIV,882467283.0,08/04/2019,1.0,NaN,NaN,NaN
2,Emídio Alves,319500,23,ADII,295150983.0,09/02/2019,3.0,NaN,NaN,NaN
3,Santos Costa,252000,64,FIIV,430169311.0,10/03/2019,2.0,NaN,NaN,NaN
4,Do Monteiro,351000,19,OPV,839424894.0,31/01/2019,3.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
320,Manoel Costa,369000,53,LOIV,801388803.0,26/11/2018,4.0,NaN,NaN,NaN
321,Gomes Machado,238500,45,OPV,131504555.0,04/12/2018,2.0,NaN,NaN,NaN
322,Alkindar Cardozo,351000,16,FIII,560973100.0,07/12/2018,1.0,NaN,NaN,NaN
323,Pereira Fazenda,418500,28,FIII,233130568.0,02/02/2019,1.0,NaN,NaN,NaN


Pode-se observar algumas colunas que não são necessárias, a coluna CEP não está em um formato desejável. Vamos fazer os ajustes necessários

In [23]:
df_clients_parquet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 325 entries, 0 to 324
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   cliente                 325 non-null    object 
 1   valor_contrato_anual    325 non-null    int64  
 2   quantidade_de_servicos  325 non-null    object 
 3   cargo_responsavel       321 non-null    object 
 4   cep                     317 non-null    float64
 5   data_inicio_contrato    317 non-null    object 
 6   nivel_de_importancia    317 non-null    float64
 7   unnamed_7               0 non-null      float64
 8   unnamed_8               0 non-null      float64
 9   unnamed_9               0 non-null      float64
dtypes: float64(5), int64(1), object(4)
memory usage: 25.5+ KB


In [24]:
# Tratamento da coluna quantidade_de_servicos
df_clients_parquet['quantidade_de_servicos'] = pd.to_numeric(df_clients_parquet['quantidade_de_servicos'], errors='coerce').astype('Int64')

# Tratamento da coluna data_inicio_contrato
df_clients_parquet['data_inicio_contrato'] = pd.to_datetime(df_clients_parquet['data_inicio_contrato'], format='%d/%m/%Y', errors='coerce')

# Tratamento da coluna cep
df_clients_parquet['cep'] = df_clients_parquet['cep'].fillna(0).astype(int).astype(str)


In [25]:
# E por fim exclusão das coluna nulas
colunas_para_remover = ['unnamed_7', 'unnamed_8', 'unnamed_9']
df_clients_parquet = df_clients_parquet.drop(columns=colunas_para_remover)

In [26]:
df_clients_parquet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 325 entries, 0 to 324
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   cliente                 325 non-null    object        
 1   valor_contrato_anual    325 non-null    int64         
 2   quantidade_de_servicos  321 non-null    Int64         
 3   cargo_responsavel       321 non-null    object        
 4   cep                     325 non-null    object        
 5   data_inicio_contrato    317 non-null    datetime64[ns]
 6   nivel_de_importancia    317 non-null    float64       
dtypes: Int64(1), datetime64[ns](1), float64(1), int64(1), object(3)
memory usage: 18.2+ KB


E por fim, vamos transformar o dataframe em parquet e carregá-lo na camada GOLD

In [27]:
df_clients_parquet.to_parquet('gold/clients.parquet')

Funcionários

In [28]:
df_employers_parquet

,id_rh,rg,cpf,ramal,estado_civil,nome_completo,login,data_de_nascimento,cep,data_de_contratacao,...,cargo,bandeira,codigos,quantidade_de_acessos,ferias_acumuladas,ferias_remuneradas,horas_extras,valores_adicionais,id_de_pessoal,id_da_area
0,1,455550390,57507179051,6482,C,Gabriel Mesquita,gabriel.mesquita,25569,20125535,38792,...,OPV,NaN,UAK1729-MCG,140,47,NaN,97,NaN,NaN,10
1,2,732355385,50388536767,6177,C,João Haddad,joão.haddad,27145,25995770,39605,...,LOI,NaN,QDI6697-POC,143,31,NaN,166,NaN,NaN,545
2,3,943205737,59035293914,7736,C,Amanda Marques Ribeiro,amanda.ribeiro,32880,37278465,39450,...,ADI,NaN,AJB6796-ZAR,110,60,NaN,188,NaN,NaN,213
3,4,968298499,51397043200,5238,C,Guilherme Nunez,guilherme.nunez,34096,49897703,40666,...,ADII,NaN,UXE5742-IZD,101,41,NaN,176,NaN,NaN,213
4,5,332164465,1296878897,5182,C,Adelino Gomes,adelino.gomes,27118,30149335,36601,...,ADII,NaN,OKI6642-TSV,35,9,NaN,121,NaN,NaN,213
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534,535,195532101,2439810843,5239,C,Bernardo da cunha,bernardo.cunha,29034,45499870,37330,...,LOII,NaN,CKP3287-UEK,55,57,NaN,88,NaN,NaN,545
535,536,486535229,4764050200,3572,C,Julia Novaes Silva,julia.silva,33916,21517794,41578,...,COII,NaN,XWA4609-XHL,139,41,NaN,44,NaN,NaN,11
536,537,981279157,54115282873,1376,S,Arthur Bispo,arthur.bispo,34170,34689811,40740,...,COII,NaN,SEA1242-VQT,60,35,NaN,4,NaN,NaN,11
537,538,464550454,16126495226,6269,S,Raianne Brum,raianne.brum,32295,31097240,41496,...,FIV,NaN,QLF8561-NLL,56,44,NaN,114,NaN,NaN,322


In [29]:
# Verificação de registros nulos por Coluna
registros_nulos_total = df_employers_parquet.isnull().sum()
print(f"Total de registros:\n{registros_nulos_total}")

Total de registros:
id_rh                                        0
rg                                           0
cpf                                          0
ramal                                        0
estado_civil                                 0
nome_completo                                0
login                                        0
data_de_nascimento                           0
cep                                          0
data_de_contratacao                          0
data_de_demissao                           436
dias_uteis_trabalhados_ano_orcamentario      0
salario_base                                 0
impostos                                     0
beneficios                                   0
vt                                           0
vr                                           0
cargo                                        0
bandeira                                   539
codigos                                      0
quantidade_de_acessos                   

In [30]:
# Excluindo as colunas que tem 100% dos registros nulos
colunas_para_remover = ['bandeira', 'id_de_pessoal', 'ferias_remuneradas']
df_employers_parquet = df_employers_parquet.drop(columns=colunas_para_remover)

# Visualizando
registros_nulos_total = df_employers_parquet.isnull().sum()
print(f"Total de registros:\n{registros_nulos_total}")

Total de registros:
id_rh                                        0
rg                                           0
cpf                                          0
ramal                                        0
estado_civil                                 0
nome_completo                                0
login                                        0
data_de_nascimento                           0
cep                                          0
data_de_contratacao                          0
data_de_demissao                           436
dias_uteis_trabalhados_ano_orcamentario      0
salario_base                                 0
impostos                                     0
beneficios                                   0
vt                                           0
vr                                           0
cargo                                        0
codigos                                      0
quantidade_de_acessos                        0
ferias_acumuladas                       

In [31]:
# Verificando a integridade dos dados, se a coluna "id_rh" tem registros distintos

# Conta a quantidade de valores únicos
registros_duplicados = df_employers_parquet[df_employers_parquet.duplicated(subset='id_rh', keep=False)]
registros_duplicados

,id_rh,rg,cpf,ramal,estado_civil,nome_completo,login,data_de_nascimento,cep,data_de_contratacao,...,beneficios,vt,vr,cargo,codigos,quantidade_de_acessos,ferias_acumuladas,horas_extras,valores_adicionais,id_da_area


A coluna id_rh poderá ser utilizada como chave, pois os registros da coluna id_rh são distintos

In [32]:
df_employers_parquet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 539 entries, 0 to 538
Data columns (total 24 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   id_rh                                    539 non-null    int64  
 1   rg                                       539 non-null    int64  
 2   cpf                                      539 non-null    int64  
 3   ramal                                    539 non-null    int64  
 4   estado_civil                             539 non-null    object 
 5   nome_completo                            539 non-null    object 
 6   login                                    539 non-null    object 
 7   data_de_nascimento                       539 non-null    int64  
 8   cep                                      539 non-null    int64  
 9   data_de_contratacao                      539 non-null    int64  
 10  data_de_demissao                         103 non-n

In [33]:
# Converter para datetime
df_employers_parquet['data_de_nascimento'] = pd.to_datetime(df_employers_parquet['data_de_nascimento'], errors='coerce').dt.strftime('%Y-%m-%d')
df_employers_parquet['data_de_contratacao'] = pd.to_datetime(df_employers_parquet['data_de_contratacao'], errors='coerce').dt.strftime('%Y-%m-%d')
df_employers_parquet['data_de_demissao'] = pd.to_datetime(df_employers_parquet['data_de_demissao'], errors='coerce').dt.strftime('%Y-%m-%d')

# Substitui vírgulas por pontos e converte para float
df_employers_parquet['impostos'] = df_employers_parquet['impostos'].str.replace(',', '.').astype(float)
df_employers_parquet['beneficios'] = df_employers_parquet['beneficios'].str.replace(',', '.').astype(float)
df_employers_parquet['vr'] = df_employers_parquet['vr'].str.replace(',', '.').astype(float)

In [34]:
df_employers_parquet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 539 entries, 0 to 538
Data columns (total 24 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   id_rh                                    539 non-null    int64  
 1   rg                                       539 non-null    int64  
 2   cpf                                      539 non-null    int64  
 3   ramal                                    539 non-null    int64  
 4   estado_civil                             539 non-null    object 
 5   nome_completo                            539 non-null    object 
 6   login                                    539 non-null    object 
 7   data_de_nascimento                       539 non-null    object 
 8   cep                                      539 non-null    int64  
 9   data_de_contratacao                      539 non-null    object 
 10  data_de_demissao                         103 non-n

In [35]:
# Anonimzação dos campos (Dados Sensíveis)
from cryptography.fernet import Fernet

# Gere uma chave para o algoritmo Fernet
chave = Fernet.generate_key()

# Crie um objeto Fernet com a chave
fernet = Fernet(chave)

# Função para encriptar o CPF
def encrypt_cpf(cpf):
    return fernet.encrypt(str(cpf).encode())

In [36]:
# Aplicando a função nos campos desejados

df_employers_parquet['rg'] = df_employers_parquet['rg'].apply(encrypt_cpf)

df_employers_parquet['cpf'] = df_employers_parquet['cpf'].apply(encrypt_cpf)

df_employers_parquet

,id_rh,rg,cpf,ramal,estado_civil,nome_completo,login,data_de_nascimento,cep,data_de_contratacao,...,beneficios,vt,vr,cargo,codigos,quantidade_de_acessos,ferias_acumuladas,horas_extras,valores_adicionais,id_da_area
0,1,b'gAAAAABlr_4iAAFFH2kj27tRn8g5EZm-Huktx4S7QWG8...,b'gAAAAABlr_4i1gHT7J-ylAlT_2WWZxLD315BPAy1ZneT...,6482,C,Gabriel Mesquita,gabriel.mesquita,1970-01-01,20125535,1970-01-01,...,4382.0,242,719.04,OPV,UAK1729-MCG,140,47,97,NaN,10
1,2,b'gAAAAABlr_4iPMNK_EAQjZBOBQUN7LbYRPDzfWZSfJ6r...,b'gAAAAABlr_4isTcndJ3atoXzKMxnFwvlc_8_CJZHwQkg...,6177,C,João Haddad,joão.haddad,1970-01-01,25995770,1970-01-01,...,1080.8,154,574.56,LOI,QDI6697-POC,143,31,166,NaN,545
2,3,b'gAAAAABlr_4iNicuRZh_1Zei76ZmLNM4qhsm3sPfxu_B...,b'gAAAAABlr_4iEqh58ptkG8Y2soCoGIIyuhV7UefmNX7I...,7736,C,Amanda Marques Ribeiro,amanda.ribeiro,1970-01-01,37278465,1970-01-01,...,3213.2,154,729.12,ADI,AJB6796-ZAR,110,60,188,NaN,213
3,4,b'gAAAAABlr_4is82qaJ-j3k8EQls4Fa5rNpq7c8ca0P6h...,b'gAAAAABlr_4iohOEfbl0h7lhdetoh7-Bpt568Q1PVOI4...,5238,C,Guilherme Nunez,guilherme.nunez,1970-01-01,49897703,1970-01-01,...,4261.0,220,524.16,ADII,UXE5742-IZD,101,41,176,NaN,213
4,5,b'gAAAAABlr_4ierwQMiwGUL5IfYDP8O_2eR2EipAfMWIQ...,b'gAAAAABlr_4if0M9u0oFpUyDrjNfdApKnOvxJbfEUokQ...,5182,C,Adelino Gomes,adelino.gomes,1970-01-01,30149335,1970-01-01,...,1019.6,176,725.76,ADII,OKI6642-TSV,35,9,121,NaN,213
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534,535,b'gAAAAABlr_4iiq8psyV7G1hiMYMn3MdQGgGwGvNkU2gh...,b'gAAAAABlr_4imgXeBFGbJppF5Casx5sU7JLeds4M1ntR...,5239,C,Bernardo da cunha,bernardo.cunha,1970-01-01,45499870,1970-01-01,...,4324.4,286,571.20,LOII,CKP3287-UEK,55,57,88,NaN,545
535,536,b'gAAAAABlr_4i7cdSd2XDJzWkPNLNxJ8oTFUGUOBvKBb9...,b'gAAAAABlr_4ikJ0V9nrVIG1M-x8mVUbS3vVXooYtOxbu...,3572,C,Julia Novaes Silva,julia.silva,1970-01-01,21517794,1970-01-01,...,1510.4,154,581.28,COII,XWA4609-XHL,139,41,44,NaN,11
536,537,b'gAAAAABlr_4i378WOzkJTV0PZHdKibe7DlLn5zGLhedp...,b'gAAAAABlr_4ihTPGClyyRfiVFKyQ7n7b03NCINqWtDRa...,1376,S,Arthur Bispo,arthur.bispo,1970-01-01,34689811,1970-01-01,...,274.4,308,517.44,COII,SEA1242-VQT,60,35,4,NaN,11
537,538,b'gAAAAABlr_4iEOPZDj9WpAk4zJ83fWoGGbWsnEYYotOh...,b'gAAAAABlr_4i1AM06fFUb_huJZlamR9WlIvpMepooVc-...,6269,S,Raianne Brum,raianne.brum,1970-01-01,31097240,1970-01-01,...,2358.2,176,715.68,FIV,QLF8561-NLL,56,44,114,NaN,322


In [37]:
#Transformando DF em Parquet e salvando na Gold
df_employers_parquet.to_parquet('gold/employers.parquet')

Base Levels

In [38]:
df_base_levels_parquet

,nivel,descricao_nivel,tempo_no_nivel,plano_de_saude,plano_odontologico,setor_responsavel,plano_de_carreira
0,JA,Estagiário,1,None,None,NaN,NaN
1,None,Sim,Não,RH Universidade,Sim,NaN,NaN
2,DB,Analista,4,None,None,NaN,NaN
3,Sim,Não,RH Empresas,Sim,None,NaN,NaN
4,GC,Coordenador,5,None,None,NaN,NaN
5,Sim,Sim,RH Empresas,Sim,None,NaN,NaN
6,ID,Gerente,10,None,None,NaN,NaN
7,Sim,Sim,RH Empresas,Não,None,NaN,NaN
8,JE,Diretor,10,None,None,NaN,NaN
9,Sim,Sim,RH Headhunter,Não,None,NaN,NaN


In [39]:
df_base_levels_parquet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   nivel               9 non-null      object 
 1   descricao_nivel     10 non-null     object 
 2   tempo_no_nivel      10 non-null     object 
 3   plano_de_saude      5 non-null      object 
 4   plano_odontologico  1 non-null      object 
 5   setor_responsavel   0 non-null      float64
 6   plano_de_carreira   0 non-null      float64
dtypes: float64(2), object(5)
memory usage: 688.0+ bytes


In [40]:
df_base_levels_parquet

,nivel,descricao_nivel,tempo_no_nivel,plano_de_saude,plano_odontologico,setor_responsavel,plano_de_carreira
0,JA,Estagiário,1,None,None,NaN,NaN
1,None,Sim,Não,RH Universidade,Sim,NaN,NaN
2,DB,Analista,4,None,None,NaN,NaN
3,Sim,Não,RH Empresas,Sim,None,NaN,NaN
4,GC,Coordenador,5,None,None,NaN,NaN
5,Sim,Sim,RH Empresas,Sim,None,NaN,NaN
6,ID,Gerente,10,None,None,NaN,NaN
7,Sim,Sim,RH Empresas,Não,None,NaN,NaN
8,JE,Diretor,10,None,None,NaN,NaN
9,Sim,Sim,RH Headhunter,Não,None,NaN,NaN


In [41]:
import numpy as np

#Coluna Nivel
#Visualiza os valores Unicos da coluna
registros_distintos_nivel = df_base_levels_parquet['nivel'].unique()
# Lista dos valores únicos
registros_distintos_nivel = ['JA', 'DB', 'GC', 'ID', 'JE']
# Persiste no fd somente os valores correspondentes
df_base_levels_parquet['nivel'] = df_base_levels_parquet['nivel'].apply(lambda x: x if x in registros_distintos_nivel else "None")

#Coluna descricao_nivel
registros_distintos_desc = df_base_levels_parquet['descricao_nivel'].unique()
registros_distintos_desc = ['Estagiário', 'Analista', 'Coordenador', 'Gerente', 'Diretor']
df_base_levels_parquet['descricao_nivel'] = df_base_levels_parquet['descricao_nivel'].apply(lambda x: x if x in registros_distintos_desc else "None")

#Coluna tempo_no_nivel
registros_distintos_temp = df_base_levels_parquet['tempo_no_nivel'].unique()
registros_distintos_temp = ['1', '4', '5', '10']
df_base_levels_parquet['tempo_no_nivel'] = df_base_levels_parquet['tempo_no_nivel'].apply(lambda x: x if x in registros_distintos_temp else np.nan)

#Coluna plano_de_saude
registros_distintos_plan = df_base_levels_parquet['plano_de_saude'].unique()
registros_distintos_plan = ['Sim', 'Não']
df_base_levels_parquet['plano_de_saude'] = df_base_levels_parquet['plano_de_saude'].apply(lambda x: x if x in registros_distintos_plan else "None")

In [42]:
# Excluindo as colunas que tem 100% dos registros nulos
colunas_para_remover = ['setor_responsavel', 'plano_de_carreira']
df_base_levels_parquet = df_base_levels_parquet.drop(columns=colunas_para_remover)

df_base_levels_parquet

In [43]:
#Transformando DF em Parquet e salvando na Gold
df_base_levels_parquet.to_parquet('gold/base_levels.parquet')